In [1]:
import pandas as pd
import numpy as np
import math
import statsmodels.formula.api as sm

In [62]:
def get_alpha(df,y_name,beta_rank = None):
    if beta_rank != None:
        df = df.loc[df["position"] == beta_rank,:]
        y_new_name = "P" + f"_{beta_rank + 1}"
        df.rename(columns = {y_name:y_new_name},inplace = True)
        y_name = y_new_name
    Excess_return=df[y_name].mean()
    Volatility = df[y_name].var()
    SharpeRatio = Excess_return / np.sqrt(Volatility)
    #print("{} portfolio's excess return is: {:.2f}%".format(y_name,Excess_return*100))
    #print("{} portfolio's volatility is: {:.2f}%".format(y_name,Volatility*10000))
    #print("{} portfolio's sharpe ratio is: {:.2f}".format(y_name,SharpeRatio))
    summary = {}
    summary["Excess_return"] = round(Excess_return*100,2)
    # n-factor model, n = 1,3,4,5
    ## CPAM alpha
    result = sm.ols(formula = f"{y_name} ~ 1 + mktrf",data = df).fit()
    #print("{} portfolio's CAPM alpha is: {:.2f}".format(y_name,result.params["Intercept"]*100))
    summary["CAPM_alpha"] = round(result.params["Intercept"]*100,2)
    ## Three-factor alpha
    result = sm.ols(formula = f"{y_name} ~ 1 + mktrf+ smb+ hml",data = df).fit()
    #print("{} portfolio's three-factor alpha is: {:.2f}".format(y_name,result.params["Intercept"]*100))
    summary["three_factor_alpha"] = round(result.params["Intercept"]*100,2)
    ## Four-factor alpha
    result = sm.ols(formula = f"{y_name} ~ 1 + mktrf+ smb+ hml+ umd",data = df).fit()
    #print("{} portfolio's four-factor alpha is: {:.2f}".format(y_name,result.params["Intercept"]*100))
    summary["four_factor_alpha"] = round(result.params["Intercept"]*100,2)
    ## Five-factor alpha
    result = sm.ols(formula = f"{y_name} ~ 1 + mktrf+ smb+ hml+ umd+ vwf",data = df,missing='drop').fit()
    #print("{} portfolio's five-factor alpha is: {:.2f}".format(y_name,result.params["Intercept"]*100))
    summary["five_factor_alpha"] = round(result.params["Intercept"]*100,2)
    summary["Volatility"] = round(Volatility * 10000,3)
    summary["SharpeRatio"] = round(SharpeRatio,2)
    summary = pd.DataFrame(summary,index = [y_name]).T
    return summary

In [3]:
#导入数据
m_ret = pd.read_csv(r"D:\学习资料\实证金融学\data_replication\CRSP_common_stock_monthly.csv")
beta  = pd.read_csv(r"D:\学习资料\实证金融学\data_replication\Result_monthly_beta.csv")
FF_Rf = pd.read_csv(r"D:\学习资料\实证金融学\data_replication\FF_monthly_Rf.csv")
BAB  = pd.read_csv(r"D:\学习资料\实证金融学\data_replication\Result_BAB_factor.csv")
FF4  = pd.read_csv(r"D:\学习资料\实证金融学\data_replication\FF_4factor.csv")
Liquid_factor = pd.read_csv(r"D:\学习资料\实证金融学\data_replication\liquidity factor.csv")

In [4]:
## 首先处理无风险数据
FF_Rf['year']=(FF_Rf['dateff']/10000).apply(int)
FF_Rf['month']=(FF_Rf['dateff']/100).apply(int)-FF_Rf['year']*100
FF_Rf.drop(columns=['dateff'],inplace=True)
#然后处理common stock数据，分离年份和月份
m_ret['year']=(m_ret['date']/10000).apply(int)
m_ret['month']=(m_ret['date']/100).apply(int)-m_ret['year']*100
m_ret.rename(columns={'PERMNO':'id'},inplace=True)
m_ret.drop(columns=['date','SHRCD'],inplace=True)
m_ret['RET'].replace(['B','C'],[np.nan,np.nan],inplace=True)
m_ret['RET']=m_ret['RET'].apply(float)
#合并m_ret和FF_Rf
stock=pd.merge(m_ret, FF_Rf, on=['year','month'], how='left')
stock=pd.merge(stock,beta,on=['id','year','month'])
del m_ret
del FF_Rf
del beta
#求stock的超额收益
stock['ret']=stock['RET']-stock['rf']
stock.drop(columns=['RET','rf'],inplace=True)
#计算BAB的超额收益
BAB['date']=BAB['year']*100+BAB['month']
BAB.drop(columns=['year','month'],inplace=True)
BAB=BAB[BAB['date']>=192904]
# 处理FF4的数据
FF4['date']=(FF4['dateff']/100).apply(int)
FF4.drop(columns=['dateff'],inplace=True)
BAB = pd.merge(BAB,FF4,on = "date", how = "left")
# 处理流动性因子数据
Liquid_factor = Liquid_factor[["DATE","PS_VWF"]]
Liquid_factor = Liquid_factor.rename(columns = {"DATE":"date","PS_VWF":"vwf"})
Liquid_factor["date"] = (Liquid_factor['date']/100).apply(int)
BAB = pd.merge(BAB,Liquid_factor,on = "date", how = "left")

In [5]:
#计算分组的beta取值(EXCHCD要求数值是1，代表NYSE的上市公司)
stock['date']=stock['year']*100+stock['month']
stock.drop(columns=['year','month'],inplace=True)
stock.dropna(subset = ["beta"],inplace = True)
stock=stock[stock['EXCHCD']==1]

In [6]:
# 计算各个日期的beta分位数
for i in range(1,10):
    bp=stock.groupby(['date'])['beta'].quantile(q=0.1*i)
    bp=pd.DataFrame(bp)
    bp.rename(columns={'beta':'bp{}'.format(i)},inplace=True)
    stock=pd.merge(stock,bp,on=['date'],how='left')

In [19]:
# 计算各个股票在各个日期的beta排序
position = np.array(stock["beta"]).reshape(len(stock),1) > np.array(stock[["bp1","bp2","bp3","bp4","bp5","bp6","bp7","bp8","bp9"]])
position = position.sum(axis = 1)
position = pd.DataFrame(position,columns = ["position"])
stock = pd.concat([stock,position],axis = 1)
stock.drop(["bp1","bp2","bp3","bp4","bp5","bp6","bp7","bp8","bp9"],axis = 1,inplace = True)

In [28]:
stock = pd.merge(stock,FF4,on = "date", how = "left")
stock = pd.merge(stock,Liquid_factor,on = "date", how = "left")

In [80]:
summaries = pd.DataFrame()
for i in range(10):
    summary = get_alpha(stock,"ret",i)
    summaries = pd.concat([summaries,summary],axis = 1)
summary_BAB = get_alpha(BAB,"BAB")
summaries = pd.concat([summaries,summary_BAB],axis = 1)

In [85]:
summaries

,P_1,P_2,P_3,P_4,P_5,P_6,P_7,P_8,P_9,P_10,BAB
Excess_return,0.710,0.780,0.880,0.880,0.970,0.99,1.010,1.020,1.070,0.990,0.720
CAPM_alpha,0.380,0.340,0.350,0.280,0.290,0.26,0.220,0.160,0.130,-0.140,0.750
three_factor_alpha,0.250,0.210,0.190,0.100,0.070,0.02,-0.030,-0.130,-0.210,-0.600,0.760
four_factor_alpha,0.260,0.220,0.220,0.150,0.140,0.12,0.100,0.050,0.040,-0.100,0.530
five_factor_alpha,0.170,0.180,0.210,0.130,0.130,0.09,0.110,0.020,0.010,-0.150,0.510
Volatility,71.458,79.816,102.061,112.974,130.934,143.20,160.182,194.656,233.564,414.699,11.303
SharpeRatio,0.080,0.090,0.090,0.080,0.080,0.08,0.080,0.070,0.070,0.050,0.220


In [84]:
summaries.to_excel(r"D:\学习资料\实证金融学\data_replication\Result_table3.xlsx")